In [3]:
import pandas as pd
import os
import urllib
import json
from tqdm import tqdm

ROOT = os.path.abspath(os.getcwd())

# Get Data

In [2]:
df = pd.read_excel(f"{ROOT}/data/meta.xlsx", sheet_name=None)["analysed_data"]
print(df)

              강산업 물품      단가 Unnamed: 2
0            강력 20kg   27800        NaN
1         영양중력분 20kg   25000        NaN
2            박력 20kg   26500        NaN
3            설탕 15kg   20200        NaN
4            꽃소금 3kg    4800        NaN
..               ...     ...        ...
400          피자스페치 대    5500        NaN
401          잠봉 500g   14100        NaN
402              NaN     NaN        NaN
403  아몬드분말버크 11.34kg  117360        NaN
404        냉동난백액 9kg   61000        NaN

[405 rows x 3 columns]


In [3]:
df.drop(["Unnamed: 2"], axis=1, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)

              강산업 물품      단가
0            강력 20kg   27800
1         영양중력분 20kg   25000
2            박력 20kg   26500
3            설탕 15kg   20200
4            꽃소금 3kg    4800
..               ...     ...
365          피자스페치 중    5200
366          피자스페치 대    5500
367          잠봉 500g   14100
368  아몬드분말버크 11.34kg  117360
369        냉동난백액 9kg   61000

[370 rows x 2 columns]


In [4]:
# 확인용 코드
print(len(df)-len(df.drop_duplicates(subset=["강산업 물품"])))



Dup = {}
Dup_idx = []
for value in df["강산업 물품"]:
    Dup[value.split(" ")[0]] = 0

for idx, value in enumerate(df["강산업 물품"]):
    if Dup[value.split(" ")[0]] == 0:
        Dup[value.split(" ")[0]] = 1
    else:
        Dup_idx.append(idx)
        Dup[value.split(" ")[0]] += 1

Dup_len = 0
for name, value in Dup.items():
    if value > 1:
        Dup_len += 1

print(Dup_len)

55
55


In [5]:
df.drop_duplicates(subset=["강산업 물품"], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)

              강산업 물품      단가
0            강력 20kg   27800
1         영양중력분 20kg   25000
2            박력 20kg   26500
3            설탕 15kg   20200
4            꽃소금 3kg    4800
..               ...     ...
310         사각케익2호받침     400
311         사각케익2호뚜껑     950
312          피자스페치 중    5200
313          피자스페치 대    5500
314  아몬드분말버크 11.34kg  117360

[315 rows x 2 columns]


In [6]:
re_idx = []
for idx, value in enumerate(df["단가"]):
    if isinstance(value, str):
        re_idx.append(idx)
        print(value)
print("--------------------------")
print("total to remove: ", len(re_idx))

보류 22쪽
보류 23쪽
보류 8
보류 3300
보류 9300
보류 7100
보류 7100
보류 7500
보류
보류 148900
보류 4500
--------------------------
total to remove:  11


In [7]:
df.drop(re_idx, inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)

              강산업 물품      단가
0            강력 20kg   27800
1         영양중력분 20kg   25000
2            박력 20kg   26500
3            설탕 15kg   20200
4            꽃소금 3kg    4800
..               ...     ...
299         사각케익2호받침     400
300         사각케익2호뚜껑     950
301          피자스페치 중    5200
302          피자스페치 대    5500
303  아몬드분말버크 11.34kg  117360

[304 rows x 2 columns]


In [125]:
ROOT = os.path.abspath(os.getcwd())
for name in tqdm(df["강산업 물품"]):
    query = urllib.parse.quote(name)

    url = f"https://openapi.naver.com/v1/search/shop.json?query={query}&display=100"
    client_id = "4OQMv3YxrKsVojrlewHN"
    client_secret = "v7mCeaUi1B"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    response = response.read().decode("utf-8")
    response = json.loads(response)

    with open(f"{ROOT}/data/{name}.json", "w") as f:
        json.dump(response, f)

100%|██████████| 304/304 [01:37<00:00,  3.13it/s]


# Analysis

In [137]:
### description
file = open(f"{ROOT}/data/description.txt", "w")

more_expensive = []
for name, value in zip(df["강산업 물품"], df['단가']):
    low_price = 100000000
    high_price = -1
    total_price = 0

    with open(f"{ROOT}/data/{name}.json", "r") as f:
        data = json.load(f)
        if 0 == len(data["items"]):
            break

        for item in data['items']:
            now_price = int(item['lprice'])
            total_price += now_price
            if high_price < now_price:
                high_price = now_price
            if low_price > now_price:
                low_price = now_price
    
    divider = len(data['items'])
    mean = total_price/divider
    diff = round(value-mean)
    if diff > 0:
        more_expensive.append(name)        
    file.write(f"\t{name}\n")
    file.write("----------------------------------------------------\n")
    file.write("diff(origin-mean):   {:+,}\n".format(diff))
    file.write("origin:              {:,}\n".format(value))
    file.write("mean:                {mean:,}   (samples: {samples})\n"
               .format(mean=round(mean), 
                       samples=divider))
    file.write("low:                 {:,}\n".format(low_price))
    file.write("high:                {:,}\n".format(high_price))
    file.write("####################################################\n")

file.close()

In [26]:
### description01
file = open(f"{ROOT}/data/description01.txt", "w")
file.write(f"품명\t\t\t단가(원)\t\t평균시세(원)\t\t단가-평균시세(원)\n")
file.write("-----------------------------------------------------------------\n")

more_expensive = []
for name, value in zip(df["강산업 물품"], df['단가']):
    low_price = 100000000
    high_price = -1
    total_price = 0

    with open(f"{ROOT}/data/{name}.json", "r") as f:
        data = json.load(f)
        if 0 == len(data["items"]):
            break

        for item in data['items']:
            now_price = int(item['lprice'])
            total_price += now_price
            if high_price < now_price:
                high_price = now_price
            if low_price > now_price:
                low_price = now_price
    
    divider = len(data['items'])
    mean = total_price/divider
    diff = round(value-mean)
    if diff > 0:
        more_expensive.append(name)        
    file.write(f"{name:25}{value:7,}{round(mean):-10,}{diff:+10,}\n")

file.close()

In [32]:
import docx

doc = docx.Document("./data/docs.docx")

for para in doc.paragraphs:
    print(para.text)
    print(len(para.text))
    print(type(para.text))
    print(para.text.split(" "))
    print("---------")


가격 비교 정리
8
<class 'str'>
['가격', '비교', '정리']
---------

0
<class 'str'>
['']
---------

0
<class 'str'>
['']
---------
품명					단가(원)	평균시세(원)	       단가 – 평균시세 (원)
41
<class 'str'>
['품명\t\t\t\t\t단가(원)\t평균시세(원)\t', '', '', '', '', '', '', '단가', '–', '평균시세', '(원)']
---------

0
<class 'str'>
['']
---------
강력 20kg				27,800    	44,217   	     	-16,417
45
<class 'str'>
['강력', '20kg\t\t\t\t27,800', '', '', '', '\t44,217', '', '', '\t', '', '', '', '', '\t-16,417']
---------
영양중력분 20kg          		25,000    	45,408   		-20,408
51
<class 'str'>
['영양중력분', '20kg', '', '', '', '', '', '', '', '', '', '\t\t25,000', '', '', '', '\t45,408', '', '', '\t\t-20,408']
---------
박력 20kg                   		26,500    	35,695    		-9,195
57
<class 'str'>
['박력', '20kg', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '\t\t26,500', '', '', '', '\t35,695', '', '', '', '\t\t-9,195']
---------
설탕 15kg                   		20,200    	33,531   		-13,331
57
<class 'str'>
['설탕', '15kg', '', '', '

In [140]:
print("total more expensive items: ", len(more_expensive))
print("-------------------------------------------------")
for item_name in more_expensive:
    print(item_name)

total more expensive items:  23
-------------------------------------------------
에멘탈치즈가는쉬레드 750g
에멘탈크림치즈 1kg
피자치즈 2.5kg
코코아분말 1kg
건포도벌크 10kg
건조크랜베리클래식 11.34kg
몽블랑생지
타르트외피
파에테포요틴 2.5kg
파인소프트T 10kg
파인소프트C 2kg
아몬드프라린 5kg
오렌지스틱 1kg
스위스밸리크림치즈 13.6kg
조은밤홀 2kg
다이스밤 7L
피스타치오 1kg
냉동난백액 9kg
냉동참쑥 2kg
드롭밀크초콜렛 5kg
드롭화이트초콜렛 5kg
홍국쌀가루 대두
베이컨 1kg


# Extra filtering

In [19]:
import re
name = "강력 20kg"
origin = 27800
total_price = 0
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if len(re.findall("유기농", item["title"])) == 0:
            total_price += int(item['lprice'])
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):,}")


강력 20kg
---------------------
mean:        39,775
origin-mean: -11,975


In [23]:
import re
name = "영양중력분 20kg"
origin = 25000
total_price = 0
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if len(re.findall("유기농", item["title"])) == 0:
            total_price += int(item['lprice'])
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):,}")

영양중력분 20kg
---------------------
mean:        42,738
origin-mean: -17,738


In [25]:
import re
name = "박력 20kg"
origin = 26500
total_price = 0
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if len(re.findall("유기농", item["title"])) == 0:
            total_price += int(item['lprice'])
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):,}")


박력 20kg
---------------------
mean:        28,911
origin-mean: -2,411


In [31]:
import re
name = "에멘탈크림치즈 1kg"
origin = 18700
total_price = 0
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if int(item["lprice"])<20000  and  int(item["lprice"])>10000\
            and  len(re.findall(name, item['title']))==0:
            total_price += int(item['lprice'])
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")


에멘탈크림치즈 1kg
---------------------
mean:        14,593
origin-mean: +4,107


In [42]:
import re
name = "건포도벌크 10kg"
origin = 68000
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")


건포도벌크 10kg
---------------------
mean:        61,958
origin-mean: +6,042
low:         58,650
high:        67,500


In [46]:
import re
name = "건조크랜베리클래식 11.34kg"
origin = 122000
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")


건조크랜베리클래식 11.34kg
---------------------
sample is 1
mean:        116,700
origin-mean: +5,300
low:         116,700
high:        116,700


In [50]:
import re
name = "조커에이"
origin = 5300
term = 5000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

조커에이
---------------------
sample is 2
mean:        5,895
origin-mean: -595
low:         1,890
high:        9,900


In [57]:
import re
name = "몽블랑생지"
origin = 107520
term = 20000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

몽블랑생지
---------------------
sample is 7
mean:        110,159
origin-mean: -2,639
low:         99,000
high:        119,500


In [58]:
import re
name = "파에테포요틴 2.5kg"
origin = 55000
term = 20000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

파에테포요틴 2.5kg
---------------------
sample is 57
mean:        55,677
origin-mean: -677
low:         49,270
high:        73,620


In [62]:
import re
name = "파인소프트T 10kg"
origin = 45400
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

파인소프트T 10kg
---------------------
sample is 42
mean:        46,751
origin-mean: -1,351
low:         42,400
high:        55,000


In [63]:
import re
name = "파인소프트C 2kg"
origin = 33100
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

파인소프트C 2kg
---------------------
sample is 38
mean:        34,318
origin-mean: -1,218
low:         24,590
high:        41,400


In [66]:
import re
name = "아몬드프라린 5kg"
origin = 142400
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

아몬드프라린 5kg
---------------------
sample is 12
mean:        140,757
origin-mean: +1,643
low:         133,100
high:        150,600


In [67]:
import re
name = "오렌지스틱 1kg"
origin = 52300
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

오렌지스틱 1kg
---------------------
sample is 43
mean:        48,906
origin-mean: +3,394
low:         45,600
high:        55,200


In [69]:
import re
name = "스위스밸리크림치즈 13.6kg"
origin = 125000
term = 50000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

스위스밸리크림치즈 13.6kg
---------------------
sample is 1
mean:        139,000
origin-mean: -14,000
low:         139,000
high:        139,000


In [72]:
import re
name = "조은밤홀 2kg"
origin = 21000
term = 5000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

조은밤홀 2kg
---------------------
sample is 13
mean:        21,325
origin-mean: -325
low:         16,900
high:        24,090


In [73]:
import re
name = "다이스밤 7L"
origin = 57000
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

다이스밤 7L
---------------------
sample is 12
mean:        53,300
origin-mean: +3,700
low:         48,000
high:        60,460


In [74]:
import re
name = "피스타치오 1kg"
origin = 34700
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

피스타치오 1kg
---------------------
sample is 29
mean:        37,691
origin-mean: -2,991
low:         32,900
high:        43,420


In [76]:
import re
name = "피스타치오커넬 100g"
origin = 11400
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

피스타치오커넬 100g
---------------------
sample is 44
mean:        13,563
origin-mean: -2,163
low:         9,500
high:        21,100


In [78]:
import re
name = "냉동난백액 9kg"
origin = 61000
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

냉동난백액 9kg
---------------------
sample is 16
mean:        59,398
origin-mean: +1,602
low:         52,000
high:        70,200


In [79]:
import re
name = "식용유 18L"
origin = 65000
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

식용유 18L
---------------------
sample is 57
mean:        63,150
origin-mean: +1,850
low:         55,100
high:        73,960


In [80]:
import re
name = "내츄럴믹스망고 250g"
origin = 12300
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

내츄럴믹스망고 250g
---------------------
sample is 41
mean:        13,692
origin-mean: -1,392
low:         2,500
high:        21,600


In [88]:
import re
name = "냉동참쑥 2kg"
origin = 17900
term = 5000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

냉동참쑥 2kg
---------------------
sample is 31
mean:        17,555
origin-mean: +345
low:         12,990
high:        22,710


In [94]:
import re
name = "드롭밀크초콜렛 5kg"
origin = 80000
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

드롭밀크초콜렛 5kg
---------------------
sample is 30
mean:        78,995
origin-mean: +1,005
low:         72,000
high:        88,400


In [95]:
import re
name = "드롭화이트초콜렛 5kg"
origin = 86300
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

드롭화이트초콜렛 5kg
---------------------
sample is 20
mean:        84,750
origin-mean: +1,550
low:         76,600
high:        96,000


In [98]:
import re
name = "카랏 스트로베리 5kg"
origin = 40200
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

카랏 스트로베리 5kg
---------------------
sample is 18
mean:        37,971
origin-mean: +2,229
low:         31,500
high:        48,700


In [99]:
import re
name = "홍국쌀가루 대두"
origin = 41800
term = 10000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

홍국쌀가루 대두
---------------------
sample is 38
mean:        41,310
origin-mean: +490
low:         32,560
high:        51,030


In [104]:
import re
name = "레몬쥬스 946ml"
origin = 5500
term = 2000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

레몬쥬스 946ml
---------------------
sample is 34
mean:        4,966
origin-mean: +534
low:         3,600
high:        7,200


In [105]:
import re
name = "필라델피아크림치즈 1.36kg"
origin = 23200
term = 5000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

필라델피아크림치즈 1.36kg
---------------------
sample is 35
mean:        24,129
origin-mean: -929
low:         18,500
high:        27,900


In [107]:
import re
name = "끼리크림치즈 1kg"
origin = 20000
term = 5000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

끼리크림치즈 1kg
---------------------
sample is 58
mean:        19,632
origin-mean: +368
low:         15,300
high:        24,000


In [108]:
import re
name = "카야쨈브라운 2.5kg"
origin = 28800
term = 1000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

카야쨈브라운 2.5kg
---------------------
sample is 2
mean:        29,385
origin-mean: -585
low:         29,000
high:        29,770


In [110]:
import re
name = "베이킹파우더포뮬러 300g"
origin = 1750
term = 5000
total_price = 0
low = 1000000000
high = -1
cnt = 0

with open(f"{ROOT}/data/{name}.json", "r") as f:
    data = json.load(f)
    for item in data["items"]:
        if (origin-term)<int(item["lprice"]) and int(item["lprice"])<(origin+term):
            now = int(item['lprice'])
            total_price += now
            if now < low:
                low = now
            if now > high:
                high = now
            cnt += 1

mean = total_price/cnt
print(name)
print("---------------------")
print(f"sample is {cnt}")
print(f"mean:        {round(mean):,}")
print(f"origin-mean: {round(origin-mean):+,}")
print(f"low:         {low:,}")
print(f"high:        {high:,}")

베이킹파우더포뮬러 300g
---------------------
sample is 26
mean:        3,552
origin-mean: -1,802
low:         2,950
high:        4,330
